In [ ]:
# Clone repo if running from Colab

#!git clone https://github.com/cipher982/marketing-mix-modeling.git
#!cp marketing-mix-modeling/funcs.py funcs.py

# Setup
## Import and Initialize

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import sys
import time
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
# get_ipython().run_line_magic('matplotlib', 'inline')

sns.color_palette("husl")
sns.set_style('darkgrid')

# import pystan
import nest_asyncio
nest_asyncio.apply()

import stan
import os
import json
import funcs

%load_ext autoreload

%autoreload 2

In [14]:
DATA_DIR = 'data/'
NUM_CHAINS = 12
SEED = 0

print(DATA_DIR)

data/


# Data Prep
## Load and Group Data
Load and re-format to a weekly grouping the datasets provided:
- Measured ad channels 
- Product sales
- Facebook
- TV



In [3]:
%%time

# Load measured data
m_df = pd.read_csv(DATA_DIR+"measured_ad_channel_data.csv")
m_df = m_df[m_df["channel"] != "Congrats  YOU FOUND ME!!!!"]

# extract weekly dates
m_df = funcs.add_week_start(m_df, 'day')

# get weekly media impressions
m_imp = m_df.groupby(["wk_strt_dt","channel"]).sum()['impressions'].reset_index()
m_imp = m_imp.pivot(index='wk_strt_dt', columns=['channel'], values=['impressions'])
m_imp = m_imp.droplevel(axis=1, level=0).reset_index()
m_imp = m_imp.fillna(value=0)
#m_imp.head(3)

# get weekly media spend
m_sp = m_df.groupby(["wk_strt_dt","channel"]).sum()['spend'].reset_index()
m_sp = m_sp.pivot(index='wk_strt_dt', columns=['channel'], values=['spend'])
m_sp = m_sp.droplevel(axis=1, level=0).reset_index()
m_sp = m_sp.fillna(value=0)
#m_sp.head(3)

# Get weekly sales
sales = pd.read_csv(DATA_DIR+"order_data.csv.gzip", compression="gzip")
sales['date'] = pd.to_datetime(sales['ORDER_DATE'])
sales['weekday'] = sales['date'].dt.weekday
sales["wk_strt_dt"] = sales['date'] - sales['weekday'] * timedelta(days=1)
sales = pd.DataFrame(sales.groupby(["wk_strt_dt"]).sum()['PRODUCT_SUBTOTAL'])
sales.columns = ['sales']

# Get Facebook data
fb = pd.read_csv(DATA_DIR+"collaborative_ad_data.csv")
fb = funcs.add_week_start(fb, "DATE")
fb = pd.DataFrame(fb.groupby(["wk_strt_dt"])['SPEND','IMPRESSIONS'].sum())
fb.columns = ['Facebook_spnd','Facebook_imps']

# Get TV data
tv = pd.read_csv(DATA_DIR+"tv_spend.csv")
tv.fillna(0, inplace=True)
tv = funcs.add_week_start(tv, 'date')
tv['tv_imps'] = tv['spend'] / tv['cost per view']
tv = pd.DataFrame(tv.groupby(['wk_strt_dt'])['tv_imps','spend'].sum())
tv.columns = ['tv_imps', 'tv_spnd']

# Create holiday data
dr = pd.date_range(start=sales.index.min(), end=sales.index.max())
hldy_df = pd.DataFrame()
hldy_df['date'] = dr

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())
hldy_df['holiday'] = hldy_df['date'].isin(holidays)

hldy_df = funcs.add_week_start(hldy_df, 'date')
hldy_df = pd.DataFrame(hldy_df.groupby(["wk_strt_dt"]).any()['holiday'])
hldy_df = hldy_df.astype(int)
print("Loaded all datasets.")

Loaded all datasets.
CPU times: user 1.76 s, sys: 128 ms, total: 1.89 s
Wall time: 1.89 s


## Combine Data

Merge the datasets together for a master df. Also add in control data for holidays.

In [4]:
%%time

# Merge all data
df = pd.merge(m_imp, m_sp, on='wk_strt_dt', suffixes=('_imps','_spnd'))
df = pd.merge(df, hldy_df, left_on='wk_strt_dt', right_index=True)
df = pd.merge(df, sales, left_on='wk_strt_dt', right_index=True)
df = pd.merge(df, fb, left_on='wk_strt_dt', right_index=True)
df = pd.merge(df, tv, left_on='wk_strt_dt', right_index=True, how='left')

# set placeholder for seasonality
df['seasonality'] = 1.0 

# ensure no 0s in the data (bugs like them)
df.fillna(value=0.0, inplace=True)
df = df.replace(to_replace=0.0, value=1.0) 

# mean-centralize: sales, numeric base_vars
hldy_cols = ['holiday']
seas_cols = ['seasonality']
me_cols = []
st_cols = []
mrkdn_cols = []

df_ctrl, sc_ctrl = funcs.mean_center_transform(
    df=df, 
    cols=['sales']+me_cols+st_cols+mrkdn_cols
)
df_ctrl = pd.concat([df_ctrl, df[hldy_cols+seas_cols]], axis=1)

# variables with pos effect on sales: economy, num_stores, sales, holiday
pos_vars = ['holiday']
X1 = df_ctrl[pos_vars].values

# variables may have either pos or neg impact on sales: seasonality
pn_vars = [seas_cols[0]]
X2 = df_ctrl[pn_vars].values

print("Created master DataFrame")

Created master DataFrame
CPU times: user 10.2 ms, sys: 38 µs, total: 10.2 ms
Wall time: 9.01 ms


# Modeling

## STAN Model (control)

Create our first simple STAN model based off just the control data

In [8]:
%%time

ctrl_data = {
    'N': len(df_ctrl), # dataset
    'K1': len(pos_vars), # len of positive only features
    'K2': len(pn_vars), # len of pos or neg features
    'X1': X1, # positive features values
    'X2': X2, # pos or neg features values
    'y': df_ctrl['sales'].values, # sales as label
    'max_intercept': min(df_ctrl['sales'])
}

ctrl_code1 = '''
data {
  int N; // number of observations
  int K1; // number of positive predictors
  int K2; // number of positive/negative predictors
  real max_intercept; // restrict the intercept to be less than the minimum y
  matrix[N, K1] X1;
  matrix[N, K2] X2;
  vector[N] y; 
}

parameters {
  vector<lower=0>[K1] beta1; // regression coefficients for X1 (positive)
  vector[K2] beta2; // regression coefficients for X2
  real<lower=0, upper=max_intercept> alpha; // intercept
  real<lower=0> noise_var; // residual variance
}

model {
  // Define the priors
  beta1 ~ normal(0, 1); 
  beta2 ~ normal(0, 1); 
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  // The likelihood
  y ~ normal(X1*beta1 + X2*beta2 + alpha, sqrt(noise_var));
}
'''
post_ctrl = stan.build(
    program_code=ctrl_code1, 
    data=ctrl_data, 
    random_seed=SEED
)

fit_ctrl = post_ctrl.sample(
    num_chains=NUM_CHAINS, 
    num_samples=500
)

fit_ctrl_result = fit_ctrl

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_390u6zhc/model_dpdspt3f.stan', line 23, column 30: Argument 0.0005 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_390u6zhc/model_dpdspt3f.stan', line 23, column 24: Argument 0.05 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).
Sampling:   0%
Sampling:   8% (1500/18000)
Sampling:  17% (3000/18000)
Sampling:  25% (4500/18000)
Sampling:  33% (6000/18000)
Sampling:  42% (7500/18000)
Sampling:  50% (9000/18000)
Sampling:  58% (10500/18000)
Sampling:  67% (12000/18000)
Sampling:  75% (13500/18000)
Sampling:  83% (15000/18000)
Sampling:  92% (16500/18000)
Sampling: 100% (18000/18000)
Sampling: 100% (18000/18000), done.
Messages received during sampling:
  Gradient evaluation took 4e-05 seconds
  1000 transitions using 10 leapfrog steps per t

CPU times: user 471 ms, sys: 217 ms, total: 688 ms
Wall time: 3.28 s


## Extract and Predict
Pull the results from the STAN model and predict a level of base sales

In [22]:
fit_ctrl_result.keys()

KeysView(<stan.Fit>
Parameters:
    beta1: (1,)
    beta2: (1,)
    alpha: ()
    noise_var: ()
Draws: 6000)

In [21]:
df_ctrl.shape

(158, 3)

In [27]:
def ctrl_model_predict(ctrl_model, df):
    pos_vars, pn_vars = ctrl_model['pos_vars'], ctrl_model['pn_vars'] 
    X1, X2 = df[pos_vars], df[pn_vars]
    beta1, beta2 = np.array(ctrl_model['beta1']), np.array(ctrl_model['beta2'])
    alpha = ctrl_model['alpha']
    print(X1.shape)
    print(beta1)
    print(X2.shape)
    print(beta2.shape)
    y_pred = np.dot(X1, beta1) + np.dot(X2, beta2) + alpha
    return y_pred

base_sales = ctrl_model_predict(base_sales_model, df_ctrl)


(158, 1)
[0.07075783 1.1677583  0.62728866 ... 1.66095706 2.15018691 1.17940757]
(158, 1)
(6000,)


ValueError: shapes (158,1) and (6000,) not aligned: 1 (dim 1) != 6000 (dim 0)

In [17]:
%%time

# Extract the model values
base_sales_model = funcs.extract_ctrl_model(
    fit_ctrl, 
    pos_vars=pos_vars, 
    pn_vars=pn_vars
)

# Predict base sales
base_sales = funcs.ctrl_model_predict(base_sales_model, df_ctrl)
# Append back to the DF
df['base_sales'] = base_sales*sc_ctrl['sales']

# Evaluate model
print('mape: ', funcs.mean_absolute_percentage_error(df['sales'], df['base_sales']))

ValueError: shapes (158,1) and (6000,) not aligned: 1 (dim 1) != 6000 (dim 0)

In [ ]:
%%time

# 2.2 Marketing Mix Model
mdip_cols = [col for col in df.columns if '_imp' in col]
mdip_cols

df_mmm, sc_mmm = funcs.mean_log1p_trandform(df, ['sales', 'base_sales'])
mu_mdip = df[mdip_cols].apply(np.mean, axis=0).values
mu_mdip[1] = 100
max_lag = 4
num_media = len(mdip_cols)
# padding zero * (max_lag-1) rows
X_media = np.concatenate((np.zeros((max_lag-1, num_media)), df[mdip_cols].values), axis=0)
X_media = np.nan_to_num(X_media)
X_ctrl = df_mmm['base_sales'].values.reshape(len(df),1)
model_data2 = {
    'N': len(df),
    'max_lag': max_lag, 
    'num_media': num_media,
    'X_media': X_media, 
    'mu_mdip': mu_mdip,
    'num_ctrl': X_ctrl.shape[1],
    'X_ctrl': X_ctrl, 
    'y': df_mmm['sales'].values
}

model_code2 = '''
functions {
  // the adstock transformation with a vector of weights
  real Adstock(vector t, row_vector weights) {
    return dot_product(t, weights) / sum(weights);
  }
}
data {
  // the total number of observations
  int<lower=1> N;
  // the vector of sales
  real y[N];
  // the maximum duration of lag effect, in weeks
  int<lower=1> max_lag;
  // the number of media channels
  int<lower=1> num_media;
  // matrix of media variables
  matrix[N+max_lag-1, num_media] X_media;
  // vector of media variables' mean
  real mu_mdip[num_media];
  // the number of other control variables
  int<lower=1> num_ctrl;
  // a matrix of control variables
  matrix[N, num_ctrl] X_ctrl;
}
parameters {
  // residual variance
  real<lower=0> noise_var;
  // the intercept
  real tau;
  // the coefficients for media variables and base sales
  vector<lower=0>[num_media+num_ctrl] beta;
  // the decay and peak parameter for the adstock transformation of
  // each media
  vector<lower=0,upper=1>[num_media] decay;
  vector<lower=0,upper=ceil(max_lag/2)>[num_media] peak;
}
transformed parameters {
  // the cumulative media effect after adstock
  real cum_effect;
  // matrix of media variables after adstock
  matrix[N, num_media] X_media_adstocked;
  // matrix of all predictors
  matrix[N, num_media+num_ctrl] X;
  
  // adstock, mean-center, log1p transformation
  row_vector[max_lag] lag_weights;
  for (nn in 1:N) {
    for (media in 1 : num_media) {
      for (lag in 1 : max_lag) {
        lag_weights[max_lag-lag+1] <- pow(decay[media], (lag - 1 - peak[media]) ^ 2);
      }
     cum_effect <- Adstock(sub_col(X_media, nn, media, max_lag), lag_weights);
     X_media_adstocked[nn, media] <- log1p(cum_effect/mu_mdip[media]);
    }
  X <- append_col(X_media_adstocked, X_ctrl);
  } 
}
model {
  decay ~ beta(3,3);
  peak ~ uniform(0, ceil(max_lag/2));
  tau ~ normal(0, 5);
  for (i in 1 : num_media+num_ctrl) {
    beta[i] ~ normal(0, 1);
  }
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  y ~ normal(tau + X * beta, sqrt(noise_var));
}
'''
print("Loading StanModel. . .")
try:
    sm2
except NameError:
    sm2 = pystan.StanModel(model_code=model_code2, verbose=True)
#sm2 = pystan.StanModel(model_code=model_code2, verbose=True)
print("Beginning sampling. . .")
fit2 = sm2.sampling(data=model_data2, iter=1000, chains=16)
fit2_result = fit2.extract()

In [ ]:
%%time

mmm = funcs.extract_mmm(
    fit2, 
    max_lag=max_lag, 
    media_vars=mdip_cols, 
    ctrl_vars=['base_sales']
)

# save_json(mmm, 'mmm1.json')
# plot media coefficients' distributions
# red line: mean, green line: median
beta_media = {}
for i in range(len(mmm['media_vars'])):
    md = mmm['media_vars'][i]
    betas = []
    for j in range(len(mmm['beta_list'])):
        betas.append(mmm['beta_list'][j][i])
    beta_media[md] = np.array(betas)

f = plt.figure(figsize=(22,18))
for i in range(len(mmm['media_vars'])):
    ax = f.add_subplot(5,3,i+1)
    md = mmm['media_vars'][i]
    x = beta_media[md]
    mean_x = x.mean()
    median_x = np.median(x)
    ax = sns.distplot(x)
    ax.axvline(mean_x, color='r', linestyle='-')
    ax.axvline(median_x, color='g', linestyle='-')
    ax.set_title(md)

In [ ]:
# decompose sales to media contribution
mc_df = funcs.mmm_decompose_contrib(mmm, df, original_sales=df['sales'])
adstock_params = mmm['adstock_params']

# calculate media contribution percentage
mc_pct, mc_pct2 = funcs.calc_media_contrib_pct(mc_df, mdip_cols, 'sales', period=52)

# mc_df.to_csv('mc_df1.csv', index=False)
# save_json(adstock_params, 'adstock_params1.json')
# pd.concat([
#     pd.DataFrame.from_dict(mc_pct, orient='index', columns=['mc_pct']),
#     pd.DataFrame.from_dict(mc_pct2, orient='index', columns=['mc_pct2'])
# ], axis=1).to_csv('mc_pct_df1.csv')

In [ ]:
%%time

model_code3 = '''
functions {
  // the Hill function
  real Hill(real t, real ec, real slope) {
  return 1 / (1 + (t / ec)^(-slope));
  }
}

data {
  // the total number of observations
  int<lower=1> N;
  // y: vector of media contribution
  vector[N] y;
  // X: vector of adstocked media spending
  vector[N] X;
}

parameters {
  // residual variance
  real<lower=0> noise_var;
  // regression coefficient
  real<lower=0> beta_hill;
  // ec50 and slope for Hill function of the media
  real<lower=0,upper=1> ec;
  real<lower=0> slope;
}

transformed parameters {
  // a vector of the mean response
  vector[N] mu;
  for (i in 1:N) {
    mu[i] <- beta_hill * Hill(X[i], ec, slope);
  }
}

model {
  slope ~ gamma(3, 1);
  ec ~ beta(2, 2);
  beta_hill ~ normal(0, 1);
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01); 
  y ~ normal(mu, sqrt(noise_var));
}
'''


In [ ]:
%%time

# train hill models for all media channels
try:
    sm3
except NameError:
    sm3 = pystan.StanModel(model_code=model_code3, verbose=True)
sm3 = pystan.StanModel(model_code=model_code3, verbose=True)
hill_models = {}
#to_train = ['dm', 'inst', 'nsp', 'auddig', 'audtr', 'vidtr', 'viddig', 'so', 'on', 'sem']
to_train = ['ASA', 'Affiliate', 'Display', 'Email', 'PLA', 
            'Search', 'Social', 'Video', 'Facebook', 'tv']
for media in to_train:
    print('training for media: ', media)
    hill_model = funcs.train_hill_model(df, mc_df, adstock_params, media, sm3)
    print("trained for media: ", media)
    hill_models[media] = hill_model

# extract params by mean
hill_model_params_mean, hill_model_params_med = {}, {}
for md in list(hill_models.keys()):
    print("extracting " + md)
    hill_model = hill_models[md]
    params1 = funcs.extract_hill_model_params(hill_model, method='mean')
    params1['sc'] = hill_model['sc']
    hill_model_params_mean[md] = params1
#     params2 = extract_hill_model_params(hill_model, method='median')
#     params2['sc'] = hill_model['sc']
#     hill_model_params_med[md] = params2
# save_json(hill_model_params_med, 'hill_model_params_med.json')
# save_json(hill_model_params_mean, 'hill_model_params_mean.json')

# evaluate model params extracted by mean
for md in list(hill_models.keys()):
    print('evaluating media: ', md)
    hill_model = hill_models[md]
    hill_model_params = hill_model_params_mean[md]
    _ = funcs.evaluate_hill_model(hill_model, hill_model_params)
# evaluate model params extracted by median
# for md in list(hill_models.keys()):
#     print('evaluating media: ', md)
#     hill_model = hill_models[md]
#     hill_model_params = hill_model_params_med[md]
#     _ = evaluate_hill_model(hill_model, hill_model_params)



In [ ]:
%%time

# Calculate overall ROAS and weekly ROAS
# - Overall ROAS = total contribution / total spending
# - Weekly ROAS = weekly contribution / weekly spending

# adstocked media spending
ms_df = pd.DataFrame()
for md in list(hill_models.keys()):
    #print("md: ",md)
    hill_model = hill_models[md]
    x = np.array(hill_model['data']['X']) * hill_model['sc']['x']
    ms_df[md+"_spnd"] = x
# ms_df.to_csv('ms_df1.csv', index=False)

roas_1y = funcs.calc_roas(mc_df, ms_df, period=52)
weekly_roas = funcs.calc_weekly_roas(mc_df, ms_df)
roas1y_df = pd.DataFrame(index=weekly_roas.columns.tolist())
roas1y_df['roas_mean'] = weekly_roas[-52:].apply(np.mean, axis=0)
roas1y_df['roas_median'] = weekly_roas[-52:].apply(np.median, axis=0)

# plot weekly ROAS distribution of past 1 year
# median: green line, mean: red line
f = plt.figure(figsize=(22,15))
for i in range(len(weekly_roas.columns)):
    md = weekly_roas.columns[i]
    ax = f.add_subplot(4,3,i+1)
    x = weekly_roas[md][-52:]
    mean_x = np.mean(x)
    median_x = np.median(x)
    ax = sns.distplot(x)
    ax.axvline(mean_x, color='r', linestyle='-', alpha=0.5)
    ax.axvline(median_x, color='g', linestyle='-', alpha=0.5)
    ax.set(xlabel=None)
    ax.set_title(md)


# Calculate mROAS
# 1. Current spending level (cur_sp) is represented by mean or median of weekly spending.    
# Next spending level (next_sp) is increasing cur_sp by 1%.
# 2. Plug cur_sp and next_sp into the Hill function:    
# Current media contribution: cur_mc = Hill(cur_sp)    
# Next-level media contribution next_mc = Hill(next_sp)    
# 3. mROAS = (next_mc - cur_mc) / (0.01 * cur_sp)


# calc mROAS of recent 1 year
mroas_1y = {}
for md in list(hill_models.keys()):
    hill_model = hill_models[md]
    hill_model_params = hill_model_params_mean[md]
    mroas_1y[md] = funcs.calc_mroas(hill_model, hill_model_params, period=52)

roas1y_df = pd.concat([
    roas1y_df[['roas_mean', 'roas_median']],
    pd.DataFrame.from_dict(mroas_1y, orient='index', columns=['mroas']),
    pd.DataFrame.from_dict(roas_1y, orient='index', columns=['roas_avg'])
], axis=1)
# roas1y_df.to_csv('roas1y_df1.csv')

roas1y_df
# **ROAS & mROAS**    
# 'roas_avg': overall ROAS = total contribution / total spending    
# 'roas_mean': mean of weekly ROAS
# 'roas_median': median of weekly ROAS    
# 'mroas': mROAS calculated based on increasing current spending level by 1%   

In [ ]:
recent_dates = df[-52:-2]['wk_strt_dt']

f = plt.figure(figsize=(22,15))
for i in range(len(weekly_roas.columns)):
    md = weekly_roas.columns[i]
    print(md)
    ax = f.add_subplot(4,3,i+1)
    x = weekly_roas[md][-52:-2]
    print(x.min())
    mean_x = np.mean(x)
    median_x = np.median(x)
    ax = sns.lineplot(x=list(range(0,50)),y=x)
    #ax.axvline(mean_x, color='r', linestyle='-', alpha=0.5)
    #ax.axvline(median_x, color='g', linestyle='-', alpha=0.5)
    ax.set(xlabel=None)
    ax.set_title(md)